In [1]:
from tqdm import tqdm
import re
import os
import json
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.agents import Tool
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.base import VectorStoreRetriever
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

from langchain.agents import initialize_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationalRetrievalChain
import pinecone
from langchain.vectorstores import Pinecone
import os
from langchain.chains import RetrievalQA

C:\Users\nauman.tasawar\AppData\Roaming\Python\Python311\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
openai_api_key = os.environ["THINK_MALL_GPT_KEY"]

PINECONE_API_KEY = os.environ["PINE_API_KEY"]

PINECONE_API_ENV = os.environ["PINE_ENV"]

In [3]:
pinecone.init(
            api_key= PINECONE_API_KEY, 
            environment= PINECONE_API_ENV
)
index_name = "think-mall"
if index_name not in pinecone.list_indexes():
    raise ValueError(
        f"No '{index_name}' index exists. You must create the index before "
        "running this notebook. Please refer to the walkthrough at "
        "'github.com/pinecone-io/examples'."  # TODO add full link
    )

In [4]:
index = pinecone.Index(index_name)
embeddings = OpenAIEmbeddings(model_name='gpt-3.5-turbo' , openai_api_key=openai_api_key)

vectordb = Pinecone.from_existing_index(index_name , embeddings)
llm = ChatOpenAI(model_name='gpt-3.5-turbo',openai_api_key=openai_api_key , temperature=0)


C:\Users\nauman.tasawar\AppData\Roaming\Python\Python311\site-packages\langchain\embeddings\openai.py:217: UserWarning: WARNING! model_name is not default parameter.
                    model_name was transferred to model_kwargs.
                    Please confirm that model_name is what you intended.
  warnings.warn(


In [5]:
retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
)
tool_desc = """Use this tool to answer user questions about the shops in Ibne Batuta Mall. If the user ask about shops information of some shopping related things then use this tool to get
the answer. This tool can also be used for follow up questions from the user."""


In [6]:
tools = [Tool(
    func=retriever.run,
    description=tool_desc,
    name='Ibne Batuta Mall'
)]

In [22]:
# memory = ConversationBufferWindowMemory(
#     memory_key="chat_history",  # important to align with agent prompt (below)
#     k=5,
#     return_messages=True
# )
memory = ConversationBufferMemory(memory_key="chat_history", return_messages= True)

In [23]:
conversational_agent = initialize_agent(
    agent='chat-conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=2,
    early_stopping_method="generate",
    memory=memory,
)

In [24]:
sys_msg = """ **Role:**
        - Embody the persona of "THINK MALL," a friendly assistant dedicated to helping users discover shops in the mall.
        - Engage users in lively conversations. Do not let the user know that you are reading any context. Maintain tone defined below.
    ** Tone **
        Generate responses in a warm, friendly, and helpful tone. \
        Expressing enthusiasm and interest in the user's input or topic. \
        Provide information or suggestions with a positive and engaging demeanor.\
      
    
    **Let's tackle this step by step:**

        1. **Determine User Query:**
        - Check if the user's query pertains to the context.
        - If not, engage with the user accordingly.
        - Do not mention that you are reading any context.

        2. **Contextual Responses:**
        - If the user's query aligns with the context, provide a relevant answer.
        - Keep responses concise, limited to 20 words.

        3. **Maintain Friendliness:**
        - Ensure a friendly tone throughout the conversation.
        
        4. **No Self-Generated Content:**
        - Strictly adhere to the provided context; avoid introducing unrelated information.

        5. **Contextual Boundaries:**
        - Only generate responses within the specified context.

        6. **Handling Unknowns:**
        - If uncertain, politely inform the user of your lack of knowledge.

        7. **Avoid Extraneous Content:**
        - Don't write code, essays, or information beyond the given context.
"""


In [25]:
prompt = conversational_agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)
conversational_agent.agent.llm_chain.prompt = prompt


In [26]:
zz = conversational_agent("where can find the gaming accessoeires")

print(zz['output'])




> Entering new AgentExecutor chain...
{
    "action": "Ibne Batuta Mall",
    "action_input": "gaming accessories"
}
Observation: Z Games, located in China Court, is a gaming specialist store that offers a wide range of gaming accessories. They aim to cater to gamers' needs in the Middle East and provide an extensive library of video games, accessories by leading brands, gaming consoles, and much more. You can contact them at 048348455.
Thought:{
    "action": "Final Answer",
    "action_input": "Z Games, located in China Court, is a gaming specialist store that offers a wide range of gaming accessories. They aim to cater to gamers' needs in the Middle East and provide an extensive library of video games, accessories by leading brands, gaming consoles, and much more. You can contact them at 048348455."
}

> Finished chain.
Z Games, located in China Court, is a gaming specialist store that offers a wide range of gaming accessories. They aim to cater to gamers' needs in the Middle East

In [27]:
zz =  conversational_agent("i want to eat pizza and then i wil buy gaming consoles")
print(zz['output'])



> Entering new AgentExecutor chain...
{
    "action": "Ibne Batuta Mall",
    "action_input": "pizza"
}
Observation: I'm sorry, but I don't have any information about pizza in the given context.
Thought:{
    "action": "Final Answer",
    "action_input": "I'm sorry, but I don't have any information about pizza or gaming consoles in the given context."
}

> Finished chain.
I'm sorry, but I don't have any information about pizza or gaming consoles in the given context.


In [13]:
zz =  conversational_agent("how can i reach them")
print(zz['output'])



> Entering new AgentExecutor chain...
{
    "action": "Ibne Batuta Mall",
    "action_input": "Z Games"
}
Observation: Z Games is a gaming specialist store located in China Court. They offer an extensive library of video games, accessories, gaming consoles, and more at competitive prices. They are open at 10:00 AM and can be reached at 048348455. They are similar to brands like ALA HAWAK, SAMSUNG, and VIRGIN MEGASTORE.
Thought:{
    "action": "Final Answer",
    "action_input": "Z Games is a gaming specialist store located in China Court. They offer an extensive library of video games, accessories, gaming consoles, and more at competitive prices. They are open at 10:00 AM and can be reached at 048348455. They are similar to brands like ALA HAWAK, SAMSUNG, and VIRGIN MEGASTORE."
}

> Finished chain.


{'input': 'how can i reach them',
 'chat_history': [HumanMessage(content='where can find the gaming accessoeires'),
  AIMessage(content="Z Games, located in China Court, is a gaming specialist store that offers a wide range of gaming accessories. They aim to cater to gamers' needs in the Middle East and provide an extensive library of video games, accessories by leading brands, gaming consoles, and much more. You can reach them at 048348455."),
  HumanMessage(content='similar shops to that'),
  AIMessage(content="Similar shops to Brands for Less in Andalusia Court include MY VAPERY, SMOKERS CENTRE, and THOMSUN PURE MUSIC. Similar shops to Athlete's Co in Egypt Court include SPORTS MARKET, COLUMBIA, and NIKE. Similar shops to The Custom Shop in India Court include KOTON, ARROW, and CELIO. Similar shops to Just Accessories in Egypt Court include ALDO ACCESSORIES and CLAIRE'S."),
  HumanMessage(content='how can i reach them'),
  AIMessage(content='Z Games is a gaming specialist store loca

[HumanMessage(content='where can find the gaming accessoeires'),
 AIMessage(content="Z Games, located in China Court, is a gaming specialist store that offers a wide range of gaming accessories. They aim to cater to gamers' needs in the Middle East and provide an extensive library of video games, accessories by leading brands, gaming consoles, and much more. You can reach them at 048348455."),
 HumanMessage(content='similar shops to that'),
 AIMessage(content="Similar shops to Brands for Less in Andalusia Court include MY VAPERY, SMOKERS CENTRE, and THOMSUN PURE MUSIC. Similar shops to Athlete's Co in Egypt Court include SPORTS MARKET, COLUMBIA, and NIKE. Similar shops to The Custom Shop in India Court include KOTON, ARROW, and CELIO. Similar shops to Just Accessories in Egypt Court include ALDO ACCESSORIES and CLAIRE'S."),
 HumanMessage(content='how can i reach them'),
 AIMessage(content='Z Games is a gaming specialist store located in China Court. They offer an extensive library of v

In [17]:
print(zz)

{'input': 'similar shops to that', 'chat_history': [HumanMessage(content='where can find the gaming accessoeires'), AIMessage(content="Z Games, located in China Court, is a gaming specialist store that offers a wide range of gaming accessories. They aim to cater to gamers' needs in the Middle East and provide an extensive library of video games, accessories by leading brands, gaming consoles, and much more. You can reach them at 048348455."), HumanMessage(content='similar shops to that'), AIMessage(content="Similar shops to Brands for Less in Andalusia Court include MY VAPERY, SMOKERS CENTRE, and THOMSUN PURE MUSIC. Similar shops to Athlete's Co in Egypt Court include SPORTS MARKET, COLUMBIA, and NIKE. Similar shops to The Custom Shop in India Court include KOTON, ARROW, and CELIO. Similar shops to Just Accessories in Egypt Court include ALDO ACCESSORIES and CLAIRE'S.")], 'output': "Similar shops to Brands for Less in Andalusia Court include MY VAPERY, SMOKERS CENTRE, and THOMSUN PURE 

In [18]:
zz =  conversational_agent("i am looking for video games")
print(zz)



> Entering new AgentExecutor chain...
{
    "action": "Ibne Batuta Mall",
    "action_input": "video games"
}
Observation: ZGames, located in China Court, is a gaming specialist store that offers an extensive library of video games, accessories, gaming consoles, and more. They aim to cater to gamers' needs in the Middle East and provide competitive prices. They are open at 10:00 AM and can be reached at 048348455.
Thought:{
    "action": "Final Answer",
    "action_input": "ZGames, located in China Court, is a gaming specialist store that offers an extensive library of video games, accessories, gaming consoles, and more. They aim to cater to gamers' needs in the Middle East and provide competitive prices. They are open at 10:00 AM and can be reached at 048348455."
}

> Finished chain.
{'input': 'i am looking for video games', 'chat_history': [HumanMessage(content='where can find the gaming accessoeires'), AIMessage(content="Z Games, located in China Court, is a gaming specialist stor

In [28]:
zz =  conversational_agent("looking for piiza shop in the mall")
print(zz)



> Entering new AgentExecutor chain...
{
    "action": "Ibne Batuta Mall",
    "action_input": "pizza shop"
}
Observation: I'm sorry, but there is no information provided about a pizza shop in the given context.
Thought:{
    "action": "Final Answer",
    "action_input": "I'm sorry, but there is no information provided about a pizza shop in the given context."
}

> Finished chain.
{'input': 'looking for piiza shop in the mall', 'chat_history': [HumanMessage(content='where can find the gaming accessoeires'), AIMessage(content="Z Games, located in China Court, is a gaming specialist store that offers a wide range of gaming accessories. They aim to cater to gamers' needs in the Middle East and provide an extensive library of video games, accessories by leading brands, gaming consoles, and much more. You can contact them at 048348455."), HumanMessage(content='i want to eat pizza and then i wil buy gaming consoles'), AIMessage(content="I'm sorry, but I don't have any information about pizz